In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
from shapely.geometry import shape, MultiPolygon
from descartes import PolygonPatch
import csv
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

In [5]:
with open("petition-jun-27-23h.json") as json_file:
    petition_old = json.load(json_file)

with open("petition-jun-29-8h.json") as json_file:
    petition_new = json.load(json_file)

total_petition_votes = 0
signatures_by_constituency = {}
for constituency in petition_old['data']['attributes']['signatures_by_constituency']:
    name = constituency['name']
    signatures_by_constituency[name] =\
    {'signatures_old': constituency['signature_count']}
    for constituency_new in petition_new['data']['attributes']['signatures_by_constituency']:
        if name == constituency_new['name']:
            signatures_by_constituency[name]['signatures_new'] = constituency_new['signature_count']
    signatures_by_constituency[name]['new_votes'] =\
        signatures_by_constituency[name]['signatures_new'] - signatures_by_constituency[name]['signatures_old']
    
signature_list = [(k, v['signatures_new'] - v['signatures_old'])
                      for (k,v) in signatures_by_constituency.items()]
signature_list.sort(key=lambda x: x[1])

print("New Votes  Constituency")
for constituency in signature_list[:10]:
    print("%5i       %s" % constituency[::-1])
print(".....      .....")
for constituency in signature_list[-10:]:
    print("%5i       %s" % constituency[::-1])

New Votes  Constituency
-3187       Bracknell
  -48       Bradford West
   16       Na h-Eileanan an Iar
   34       Banff and Buchan
   34       Glasgow North East
   35       Glenrothes
   36       Motherwell and Wishaw
   36       Easington
   40       Caithness, Sutherland and Easter Ross
   41       Orkney and Shetland
.....      .....
  681       Streatham
  684       Hampstead and Kilburn
  688       Dulwich and West Norwood
  692       Vauxhall
  708       Holborn and St Pancras
  741       Hackney South and Shoreditch
  753       Hackney North and Stoke Newington
  755       Hornsey and Wood Green
  757       Islington North
  991       Bristol West


In [8]:
conversions = []
with open("wards_to_lads.csv") as csv_file:
    reader = csv.reader(csv_file)
    for ii, line in enumerate(reader):
        if ii != 0:
            ward = line[0]
            district = line[1]
            conversions.append((ward, district))

signatures_by_lad = {}
for (ward, count) in signatures_by_constituency.items():
    matches = 0
    for (ward_match, district) in conversions:
        if ward == ward_match:
            matches += 1
        if district not in signatures_by_lad:
            signatures_by_lad[district] = 0
    
    for (ward_match, district) in conversions:
        if ward == ward_match:
            signatures_by_lad[district] += count['new_votes']/matches

signature_list = [[k, v] for (k, v) in signatures_by_lad.items()]
signature_list.sort(key=lambda x: x[1])

# # tidying
# for constituency in signature_list:
#     if constituency[0] == "Na h-Eileanan Siar":
#         constituency[0] = "Eilean Siar"
        

print("Votes  Constituency")
for constituency in signature_list[:10]:
    print("%5i %s" % tuple(constituency[::-1]))
print("..... .....")
for constituency in signature_list[-10:]:
    print("%5i %s" % tuple(constituency[::-1]))

Votes  Constituency
-2563 Bracknell Forest
  -79 Wokingham
   16 Na h-Eileanan Siar
   18 Orkney Islands
   22 Shetland Islands
   24 Moyle
   25 Ballymoney
   28 Larne
   31 Carrickfergus
   34 Boston
..... .....
 1286 Haringey
 1327 Manchester
 1420 Islington
 1423 Southwark
 1493 Hackney
 1541 Leeds
 1803 Lambeth
 1828 Wandsworth
 1848 Birmingham
 1957 Bristol, City of


In [10]:
def mercator(p):
    radius = 6378137
    out = [0, 0]
    out[0] = np.pi * radius * p[0] / 180;
    out[1] = radius * np.log(np.tan((45 + p[1]/2)*np.pi/180));

    return out

In [11]:
# Load in Geo data

with open("geo_eng_lad.json") as json_file:
    lads = json.load(json_file) 

feature_collection = lads['features']
feature = feature_collection[0]

# convert to Mercator
for feature in feature_collection:
    p = feature['geometry']
    if p['type'] == "Polygon":
        for polygon in p['coordinates']:
            for point in polygon:
                point[:] = mercator(point)
    
    elif p['type'] == "MultiPolygon":
        for polygons in p['coordinates']:
            for polygon in polygons:
                for point in polygon:
                    point[:] = mercator(point)



In [14]:
min_votes = 0
max_votes = 2000
def get_color(votes):
    if votes < 0: return (0, 0, 1)
    c = (votes)/(max_votes)
    return (1, 1, 1 - c)

# Convert to patches

patches = []
names = []
minx = np.Inf
miny = np.Inf
maxx = -np.Inf
maxy = -np.Inf


key_exists = []

england_names = []
for feature in feature_collection:
    name = feature['properties']['LAD13NM']
    if name == "Eilean Siar": name = "Na h-Eileanan Siar"
    england_names.append(name)
    p = shape(feature['geometry'])
    try:
        votes = signatures_by_lad[name]
        key_exists.append(name)
    except KeyError:
        votes = 0
    if isinstance(p, MultiPolygon):
        for poly in p:
            lminx, lminy, lmaxx, lmaxy = p.bounds
            if lminx < minx: minx = lminx
            if lminy < miny: miny = lminy
            if lmaxx > maxx: maxx = lmaxx
            if lmaxy > maxy: maxy = lmaxy
            patches.append(PolygonPatch(poly, fc=get_color(votes), ec='#555555', lw=0.2, alpha=1., zorder=1))
    else:
        patches.append(PolygonPatch(p, fc=get_color(votes), ec='#555555', lw=0.2, alpha=1., zorder=1))
        lminx, lminy, lmaxx, lmaxy = p.bounds
        if lminx < minx: minx = lminx
        if lminy < miny: miny = lminy
        if lmaxx > maxx: maxx = lmaxx
        if lmaxy > maxy: maxy = lmaxy


In [17]:
fig = plt.figure()
ax = fig.add_subplot(111)

w, h = maxx - minx, maxy - miny
xmin, xmax = minx - 0.2 * w, maxx + 0.2 * w
ymin, ymax = miny - 0.2 * h, maxy + 0.2 * h
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
ax.set_aspect(1)

ax.add_collection(PatchCollection(patches, match_original=True))
ax.set_xticks([])
ax.set_yticks([])

#scale
delta = h/20
x = 0.8*delta
ax.add_patch(Rectangle((xmin + w, ymin + 5*delta), x, x, fc=get_color(2000)))
ax.text(xmin + w + delta, ymin + 5*delta, "2000")
ax.add_patch(Rectangle((xmin + w, ymin + 4*delta), x, x, fc=get_color(1000)))
ax.text(xmin + w + delta, ymin + 4*delta, "1000")
ax.add_patch(Rectangle((xmin + w, ymin + 3*delta), x, x, fc=get_color(500)))
ax.text(xmin + w + delta, ymin + 3*delta, "500")
ax.add_patch(Rectangle((xmin + w, ymin + 2*delta), x, x, fc=get_color(0)))
ax.text(xmin + w + delta, ymin + 2*delta, "0")
ax.add_patch(Rectangle((xmin + w, ymin + 1*delta), x, x, fc=get_color(-1)))
ax.text(xmin + w + delta, ymin + 1*delta, "<0")

ax.text(xmin + 0.2*w, ymax - 0.2*w, "New Petition Signatures")
ax.text(xmin + delta, ymin + delta, "N. Lampe, 2016", color="gray", fontsize="x-small")

fig.savefig("new_signatures.png", bbox_inches="tight", dpi=300)
fig.savefig("new_signatures.svg", bbox_inches="tight")

In [16]:
signature_list = [[k, v] for (k, v) in signatures_by_lad.items() if k in england_names]
signature_list.sort(key=lambda x: x[1])
        

print("Votes  Constituency")
for constituency in signature_list[:10]:
    print("%5i %s" % tuple(constituency[::-1]))
print("..... .....")
for constituency in signature_list[-10:]:
    print("%5i %s" % tuple(constituency[::-1]))

Votes  Constituency
-2563 Bracknell Forest
  -79 Wokingham
   34 Boston
   38 Isles of Scilly
   48 Bolsover
   49 Barrow-in-Furness
   51 Hartlepool
   56 Cannock Chase
   58 Tamworth
   62 Corby
..... .....
 1286 Haringey
 1327 Manchester
 1420 Islington
 1423 Southwark
 1493 Hackney
 1541 Leeds
 1803 Lambeth
 1828 Wandsworth
 1848 Birmingham
 1957 Bristol, City of
